In [1]:
!pip install accelerate evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 121.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
!pip install transformers==4.45.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [3]:
!unzip XLM-R_base_nli_100k_2e.zip

Archive:  XLM-R_base_nli_100k_2e.zip
   creating: XLM-R_base_nli_100k_2e/
  inflating: XLM-R_base_nli_100k_2e/model.safetensors  
  inflating: XLM-R_base_nli_100k_2e/config.json  
  inflating: XLM-R_base_nli_100k_2e/special_tokens_map.json  
  inflating: XLM-R_base_nli_100k_2e/tokenizer.json  
  inflating: XLM-R_base_nli_100k_2e/sentencepiece.bpe.model  
  inflating: XLM-R_base_nli_100k_2e/tokenizer_config.json  


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load model and tokenizer
model_path = "/content/XLM-R_base_nli_100k_2e"  # change this to wherever your model is saved
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()  # evaluation mode

# Labels according to XNLI format
label_map = {
    0: "entailment",
    1: "neutral",
    2: "contradiction"
}

def predict_nli(premise, hypothesis):
    # Tokenize input
    inputs = tokenizer(premise, hypothesis, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Run through model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        predicted_label = torch.argmax(probs, dim=-1).item()

    return label_map[predicted_label], probs.squeeze().tolist()


In [6]:
# Kannada example
premise = "ಅವನು ಒಂದು ಪುಸ್ತಕವನ್ನು ಓದುತ್ತಿದ್ದನು."
hypothesis = "ಅವನು ಅಧ್ಯಯನ ಮಾಡುತ್ತಿದ್ದನು."

label, prob = predict_nli(premise, hypothesis)
print(f"Prediction: {label}")
print(f"Confidence scores: {prob}")


Prediction: entailment
Confidence scores: [0.8100453615188599, 0.1714390218257904, 0.018515529111027718]


In [9]:
# French example
premise = "ಅವನಿಗೂ ಹೋಗಲು ಮನಸ್ಸಾಗಲಿಲ್ಲ"
hypothesis = "ಈ ಆಮಂತ್ರಣವನ್ನು ಸ್ವೀಕರಿಸಿದವರಲ್ಲಿ ಅವನು ಮೊದಲಿಗನಾಗಿದ್ದು, ಆ ಅನುಭವವನ್ನು ಆನಂದಿಸಿದವನಾಗಿದ್ದನು"

label, prob = predict_nli(premise, hypothesis)
print(f"Prediction: {label}")
print(f"Confidence scores: {prob}")


Prediction: neutral
Confidence scores: [0.012540506199002266, 0.511066198348999, 0.47639328241348267]
